In [1]:
import numpy as ny
import keras as k
from matplotlib import pyplot

def getLRLU():
    return k.layers.LeakyReLU(alpha = 0.2)

def getDT():
    return k.layers.Dropout(0.4)

def getC2DT():
    return k.layers.Conv2DTranspose(128, (4, 4), strides = (2, 2), padding = 'same')

def getoptA():
    return k.optimizers.Adam(lr = 0.0002, beta_1 = 0.5)

def dd():
	mdiscr = k.models.Sequential()
	mdiscr.add(k.layers.Conv2D(64, (3,3), strides=(2, 2), padding ='same', input_shape=(28, 28, 1)))
	mdiscr.add(getLRLU())
	mdiscr.add(getDT())
	mdiscr.add(k.layers.Conv2D(64, (3,3), strides=(2, 2), padding ='same'))
	mdiscr.add(getLRLU())
	mdiscr.add(getDT())
	mdiscr.add(k.layers.Flatten())
	mdiscr.add(k.layers.Dense(1, activation = 'sigmoid'))
	ref = getoptA()
	mdiscr.compile(loss = 'binary_crossentropy', optimizer = ref, metrics = ['accuracy'])
	return mdiscr

def dg(ldim):
	mdeger = k.models.Sequential()
	nn = 128 * 7 * 7
	mdeger.add(k.layers.Dense(nn, input_dim = ldim))
	mdeger.add(getLRLU())
	mdeger.add(k.layers.Reshape((7, 7, 128)))
	mdeger.add(getC2DT())
	mdeger.add(getLRLU())
	mdeger.add(getC2DT())
	mdeger.add(getLRLU())
	mdeger.add(k.layers.Conv2D(1, (7,7), activation = 'sigmoid', padding = 'same'))
	return mdeger

def dgan(mg, md):
	md.trainable = False
	mgan = k.models.Sequential()
	mgan.add(mg)
	mgan.add(md)
	ref = getoptA()
	mgan.compile(loss = 'binary_crossentropy', optimizer = ref)
	return mgan

def lrs():
	(trainX, _), (_, _) = k.datasets.mnist.load_data()
	lrss = ny.expand_dims(trainX, axis=-1)
	lrss = lrss.astype('float32')
	lrss = lrss / 255.0
	return lrss

def grs(ds, nss):
	grss = ny.random.randint(0, ds.shape[0], nss)
	gr = ds[grss]
	g = ny.ones((nss, 1))
	return gr, g

def glp(ldim, nss):
	glpi = ny.random.randn(ldim * nss)
	glpi = glpi.reshape(nss, ldim)
	return glpi

def gfs(mg, ldim, nss):
	gfsi = glp(ldim, nss)
	gf = mg.predict(gfsi)
	g = ny.zeros((nss, 1))
	return gf, g

def spl(ex, ep, k = 10):
	for i in range(k * k):
		pyplot.subplot(k, k, 1 + i)
		pyplot.axis('off')
		pyplot.imshow(ex[i, :, :, 0], cmap = 'gray_r')
	fn = 'gen%03d.png' % (ep + 1)
	pyplot.savefig(fn)
	pyplot.close()

def sp(ep, mg, md, ds, ldim, nss = 100):
	spxr, spyr = grs(ds, nss)
	_, racc = md.evaluate(spxr, spyr, verbose = 0)
	spxf, spyf = gfs(mg, ldim, nss)
	_, facc = md.evaluate(spxf, spyf, verbose = 0)
	print(str(racc) + " " + str(facc))
	spl(spxf, ep)
	fn = 'gen%03d.h5' % (ep + 1)
	mg.save(fn)

def t(mg, md, mgan, ds, ldim, nep = 2, nb = 8192):
	bpe = int(ds.shape[0] / nb)
	hb = int(nb / 2)
	for i in range(nep):
		for j in range(bpe):
			txr, tyr = grs(ds, hb)
			txf, tyf = gfs(mg, ldim, hb)
			tx = ny.vstack((txr, txf))
			ty = ny.vstack((tyr, tyf))
			dl, _ = md.train_on_batch(tx, ty)
			txgan = glp(ldim, nb)
			tygan = ny.ones((nb, 1))
			gl = mgan.train_on_batch(txgan, tygan)
			print(str(i + 1) + ", " + str(j + 1) +"/" + str(bpe) + ", " + str(dl) + ", " + str(gl))
		if (i + 1) % 1 == 0:
			sp(i, mg, md, ds, ldim)

ldim = 100
md = dd()
mg = dg(ldim)
mgan = dgan(mg, md)
ds = lrs()
t(mg, md, mgan, ds, ldim)

KeyboardInterrupt: 